In [1]:
import pandas as pd
from docx import Document
import re
import pandas as pd
import torch
from transformers import BigBirdTokenizer,BigBirdModel,BigBirdForSequenceClassification
from torch import Tensor
import numpy as np
from collections import Counter
# from typing import list

In [2]:
# Data_csv = pd.read_csv("/home/ubuntu/working_directory/Bert_experimentation/new_dataframe_50000_100000.csv")
Data_csv = pd.read_csv("/home/ubuntu/cat_poc/llms/final_data.csv")
# Data_csv.drop('Unnamed: 0', axis=1, inplace=True)
Data_csv

,filename,text,level
0,15031-4983-FullBook.docx,"Learner Choice, Learning Voice\n\n\nLearner Vo...",Level 1
1,15031-4984-FullBook.docx,\n\nExistentialism: A Philosophical Inquiry\n\...,Level 1
2,15031-4985-FullBook.docx,"""The editors of this volume are the top practi...",Level 1
3,15031-4986-FullBook.docx,Black Power Music!\n\nBlack Power Music!: Prot...,Level 1
4,15031-4989-FullBook.docx,Home\n\n\nHome articulates a ‘critical geograp...,Level 1
...,...,...,...
1231,15032-5774-FullBook.docx,The Japanese LGBTQ+ Community in the World\n\n...,Level 3
1232,15032-5789-FullBook.docx,The Acquisition of English Grammar and Phonolo...,Level 3
1233,15032-5829-FullBook.docx,"\n‘New concepts, new words for them, new actio...",Level 3
1234,15032-5843-FullBook.docx,Translating Rumi into the West\n\n\nFocusing o...,Level 3


In [3]:
level_counts = Data_csv['level'].value_counts()
print(level_counts)

Level 1    980
Level 2    191
Level 3     65
Name: level, dtype: int64


In [4]:
sample_df = Data_csv.groupby("level").apply(lambda x: x.sample(60))

In [5]:
sample_df

filename  \
level                                    
Level 1 12    15031-5006-FullBook.docx   
        397   15032-5263-FullBook.docx   
        747   15032-5692-FullBook.docx   
        132   15031-5215-FullBook.docx   
        854   15032-5852-FullBook.docx   
...                                ...   
Level 3 1210  15031-5420-FullBook.docx   
        1174  15031-5074-FullBook.docx   
        1221  15032-5447-FullBook.docx   
        1222  15032-5474-FullBook.docx   
        1215  15031-5541-FullBook.docx   

                                                           text    level  
level                                                                     
Level 1 12    "Kenneth Harrow’s book innovatively places sci...  Level 1  
        397   Multidisciplinary Perspectives on Representati...  Level 1  
        747   Strategic Portfolio Management\n\nThis book pr...  Level 1  
        132   \n\nPromoting Your Voice on School Safety: \nA...  Level 1  
        854   Making Democratic Theory Democratic\nThis book...  Level 1  
...                                                         ...      ...  
Level 3 1210  A Dictionary of High Frequency Function Words ...  Level 3  
        1174  The Social Archaeology of Late Second Temple J...  Level 3  
        1221  Singapore Mandarin Grammar I\n\nAs the first v...  Level 3  
        1222  "This book takes a helpful evidence-based appr...  Level 3  
        1215  <<halftitle page i – mandatory element>>\nRout...  Level 3  

[180 rows x 3 columns]

In [6]:
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base')
model = BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base', 
                                                         num_labels=3)

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassifica

In [ ]:
def split_text_into_chunks(text, stride, chunk_length, min_chunk_length):
    # Split the text into sentences
    sentences = re.split(r"(?<=\.|\?|\!)\s", text)
    
    chunks = []
    current_chunk = ""
    current_length = 0
    
    for sentence in sentences:
        sentence_length = len(sentence)
        # If adding the current sentence to the chunk will exceed the chunk length, start a new chunk
        if current_length + sentence_length > chunk_length:
            if current_length >= min_chunk_length:
                # last_element = my_list[-1]
                # sub_sentences = chunks[-1]
                if chunks:
                    sub_sentences = re.split(r"(?<=\;|\,|\.|\?|\!)\s", chunks[-1])
                else:
                    sub_sentences = re.split(r"(?<=\;|\,|\.|\?|\!)\s", sentence)

                sub_sentences = re.split(r"(?<=\;|\,|\.|\?|\!)\s", sentence)
                for a, sub_sentence in enumerate(sub_sentences):
                    sub_sentence_length = len(sub_sentence)
                    if current_length + sub_sentence_length > chunk_length:
                        chunks.append(current_chunk.strip())
                        current_chunk = ""
                        current_length = 0
                    current_chunk += sub_sentence + " "
                    current_length += sub_sentence_length
                    # current_length += len(sentence)
                    # if current_length >= stride:
                    #     remove_index = a + 1
                    #     break

                chunks.append(current_chunk.strip())
                current_chunk = ""
                current_length = 0
        current_chunk += sentence + " "
        current_length += sentence_length
    if current_length >= min_chunk_length:
        chunks.append(current_chunk.strip())
    return chunks


In [ ]:
long_string = Data_csv.loc[1171]['text'] 
characters = [".", "?", "!"]
character_indices = [i for i, char in enumerate(long_string) if char in characters]

print(len(character_indices))
sentences = re.split(r"(?<=\.|\?|\!)\s", long_string)
print(len(sentences))

In [ ]:
label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
for idx in range(sample_df.shape[0]):
    x = sample_df["text"].iloc[idx]
    sentences = re.split(r'(?<=[.!?])\s+', x)
    long_string = ' '.join(sentences)
    label = label2id[sample_df.iloc[idx]["level"]]
    chunks = split_text_into_chunks(long_string, stride, chunk_length, min_chunk_length)
    trimmed_chunks = [string[:512] for string in chunks]
    for i in trimmed_chunks:
        if len(i) > 512:
            print(len(i))
    # print(len(chunks))

In [ ]:
long_string = Data_csv.loc[1171]['text']  # Your long string
print(len(long_string))
stride = 386 # Stride value
chunk_length = 510  # Maximum chunk length
min_chunk_length = 250 # Minimum chunk length
chunks = split_text_into_chunks(long_string, stride, chunk_length, min_chunk_length)
# new_chunks = build_overlapping_chunks(chunks, chunk_length, stride)
print(len(chunks))

In [ ]:
def split_text_into_chunks(text, stride, chunk_length, min_chunk_length):
    sentences = re.split(r"(?<=\.|\?|\!)\s", text)
    chunks = []
    current_chunk = " "
    for sentence in sentences:
        sen_len = len(sentence.split())
        curr_chunk_len = len(current_chunk.split())
        if sen_len + curr_chunk_len > 400:
            chunks.append(current_chunk)
            prev_chunk = chunks[-1]
            sub_sentences = re.split(r"(?<=\.|\?|\!)\s", prev_chunk)
            temp_buffer = " "
            for sub_sentece in reversed(sub_sentences):
                sub_sen_len = len(sub_sentece.split())
                temp_buffer_len = len(temp_buffer.split())
                if temp_buffer_len + sub_sen_len > 128:
                    current_chunk = temp_buffer
                    break
                temp_buffer = sub_sentece + temp_buffer
            # current_chunk += temp_buffer
        current_chunk += sentence
    return chunks

    

In [ ]:
long_string = Data_csv.loc[1171]['text'] 
# sentences = re.split(r"(?<=\.|\?|\!)\s", long_string)
# print(len(sentences))
chunks = split_text_into_chunks(long_string, stride, chunk_length, min_chunk_length)
print(chunks[0])
print("__________")
print(chunks[1])
print("__________")
print(chunks[2])


In [ ]:
long_string = Data_csv.loc[1171]['text'] 
# sentences = re.split(r"(?<=\.|\?|\!)\s", long_string)
# print(len(sentences))
chunks = split_text_into_chunks(long_string, stride, chunk_length, min_chunk_length)

tokenized_chunks = []
for chunk in chunks[0:1]:
    print("chunk_length",len(chunk.split()))
    tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
    print(len(tokens['input_ids'][0]))
    print(chunk)
    print(tokens['input_ids'])
    tokenized_chunks.append(tokens)

In [ ]:
def split_text_into_chunks(text, stride, chunk_length, min_chunk_length):
    sentences = re.split(r"(?<=\.|\?|\!)\s", text)
    chunks = []
    current_chunk = " "
    for sentence in sentences:
        sen_len = len(sentence)
        curr_chunk_len = len(current_chunk)
        if sen_len + curr_chunk_len > chunk_length:
            chunks.append(current_chunk)
            prev_chunk = chunks[-1]
            sub_sentences = re.split(r"(?<=\.|\?|\!)\s", prev_chunk)
            temp_buffer = " "
            for sub_sentece in reversed(sub_sentences):
                sub_sen_len = len(sub_sentece)
                temp_buffer_len = len(temp_buffer)
                if temp_buffer_len + sub_sen_len > stride:
                    current_chunk = temp_buffer
                    break
                temp_buffer = sub_sentece + temp_buffer
            # current_chunk += temp_buffer
        current_chunk += sentence
    return chunks


    

In [ ]:
def split_text_into_chunks(text, stride, chunk_length, min_chunk_length):
    sentences = re.split(r"(?<=\.|\?|\!|\,|\;|\n)\s", text)
    chunks = []
    current_chunk = " "
    for sentence in sentences:
        sen_len = len(sentence.split())
        curr_chunk_len = len(current_chunk.split())
        if sen_len + curr_chunk_len > 250:
            chunks.append(current_chunk)
            prev_chunk = chunks[-1]
            sub_sentences = re.split(r"(?<=\.|\?|\!|\,|\;|\n)\s", prev_chunk)
            temp_buffer = " "
            for sub_sentece in reversed(sub_sentences):
                sub_sen_len = len(sub_sentece.split())
                temp_buffer_len = len(temp_buffer.split())
                if temp_buffer_len + sub_sen_len > 64:
                    current_chunk = temp_buffer
                    break
                temp_buffer = sub_sentece + temp_buffer
            # current_chunk += temp_buffer
        current_chunk += sentence
    return chunks

    

In [ ]:
!pip install tiktoken

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 512,
    chunk_overlap  = 256,
    length_function = len,
)
long_string = Data_csv.loc[1171]['text'] 

texts = text_splitter.create_documents([long_string])
print(texts[0])
print("------------")
print(texts[1])
print("------------")
print(texts[2])

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter_1 = TokenTextSplitter(chunk_size=300, chunk_overlap=128)

text_splitter_2 = TokenTextSplitter(chunk_size=150, chunk_overlap=64)

long_string = Data_csv.loc[1171]['text'] 
texts = text_splitter.split_text(long_string)
# print(texts[0])
# print("------------")
# print(texts[1])
# print("------------")
# print(texts[2])

In [8]:
def split_tokens_into_smaller_chunks(input_id: Tensor,att_mask: Tensor, chunk_size: int, stride: int, minimal_chunk_length: int):
    input_id_chunks = [input_id[i : i + chunk_size] for i in range(0, len(input_id), stride)]
    mask_chunks = [att_mask[i : i + chunk_size] for i in range(0, len(att_mask), stride)]
    if len(input_id_chunks) > 1:
        # ignore chunks with less than minimal_length number of tokens
        input_id_chunks = [x for x in input_id_chunks if len(x) >= minimal_chunk_length]
        mask_chunks = [x for x in mask_chunks if len(x) >= minimal_chunk_length]
    return input_id_chunks, mask_chunks

# from typing import List

# def split_tokens_into_smaller_chunks(input_ids: List[int], attention_mask: List[int], chunk_size: int, stride: int, minimal_chunk_length: int):
#     input_id_chunks = [input_ids[i : i + chunk_size] for i in range(0, len(input_ids), stride)]
#     mask_chunks = [attention_mask[i : i + chunk_size] for i in range(0, len(attention_mask), stride)]
#     if len(input_id_chunks) > 1:
#         # ignore chunks with less than minimal_length number of tokens
#         input_id_chunks = [x for x in input_id_chunks if len(x) >= minimal_chunk_length]
#         mask_chunks = [x for x in mask_chunks if len(x) >= minimal_chunk_length]
#     return input_id_chunks, mask_chunks

def add_special_tokens_at_beginning_and_end(input_id_chunks, mask_chunks) -> None:
    """
    Adds special CLS token (token id = 101) at the beginning.
    Adds SEP token (token id = 102) at the end of each chunk.
    Adds corresponding attention masks equal to 1 (attention mask is boolean).
    """
    for i in range(len(input_id_chunks)):
        # adding CLS (token id 101) and SEP (token id 102) tokens
        input_id_chunks[i] = torch.cat([Tensor([101]), input_id_chunks[i], Tensor([102])])
        # adding attention masks  corresponding to special tokens
        mask_chunks[i] = torch.cat([Tensor([1]), mask_chunks[i], Tensor([1])])

def add_padding_tokens(input_id_chunks, mask_chunks) -> None:
    """Adds padding tokens (token id = 0) at the end to make sure that all chunks have exactly 512 tokens."""
    for i in range(len(input_id_chunks)):
        # get required padding length
        pad_len = 512 - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([input_id_chunks[i], Tensor([0] * pad_len)])
            mask_chunks[i] = torch.cat([mask_chunks[i], Tensor([0] * pad_len)])

# def add_padding_tokens(input_id_chunks, mask_chunks) -> None:
#     """Adds padding tokens (token id = 0) at the end to make sure that all chunks have exactly 512 tokens."""
#     for i in range(len(input_id_chunks)):
#         # get required padding length
#         pad_len = 512 - len(input_id_chunks[i])
#         # check if list length satisfies required chunk size
#         if pad_len > 0:
#             # if padding length is more than 0, we must add padding
#             input_id_chunks[i] += [0] * pad_len
#             mask_chunks[i] += [0] * pad_len

def stack_tokens_from_all_chunks(input_id_chunks, mask_chunks):
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)
    return input_ids.long(), attention_mask.int()

In [9]:
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn.functional as F
from langchain.text_splitter import TokenTextSplitter
text_splitter_1 = TokenTextSplitter(chunk_size=300, chunk_overlap=128)

text_splitter_2 = TokenTextSplitter(chunk_size=150, chunk_overlap=64)

new_chunk_df = pd.DataFrame()
total_chunks = []  
total_att_mask = [] 
total_fnames = []
total_labels = []
total_chunk_counts = []

label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
chunks_count = 1
chunk_length = 2000
stride = 500
min_chunk_length = 256
length_total_labels = 0

chunk_len_list = []
token_len_list = []
for idx in range(sample_df.shape[0]):
# for idx in range(2):
    x = sample_df["text"].iloc[idx]
    # sentences = re.split(r'(?<=[.!?])\s+', x)
    # long_string = ' '.join(sentences)
    label = label2id[sample_df.iloc[idx]["level"]]
    # chunks = split_text_into_chunks(x, stride, chunk_length, min_chunk_length)
    chunks = text_splitter_1.split_text(x)
    # length_total_labels += len(chunks)
    tokenized_chunks = []
    for chunk in chunks:
        # print("chunk_length",len(chunk.split()))
        chunk_len_list.append(len(chunk.split()))
        
        tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
        if len(tokens['input_ids'][0]) > 512:
            sub_chunks = text_splitter_2.split_text(chunk)
            print(len(sub_chunks))
            for i in sub_chunks:
                tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
                tokenized_chunks.append(tokens)
            # print(len(tokens['input_ids'][0]))
        else:
            token_len_list.append(len(tokens['input_ids'][0]))
            tokenized_chunks.append(tokens)
    input_id_chunks = []
    mask_chunks = []
    for chunk in tokenized_chunks:
        input_id_chunks.extend(chunk["input_ids"])
        mask_chunks.extend(chunk["attention_mask"])

    add_padding_tokens(input_id_chunks, mask_chunks)
    label_vec = [label] * len(input_id_chunks)
    total_chunks.extend((input_id_chunks))
    total_att_mask.extend((mask_chunks))
    total_labels.extend((label_vec))


In [ ]:
for i in total_chunks:
    if len(i) > 512:
        print(len(i))

In [ ]:
len(total_labels)

In [ ]:
len(total_chunks)

In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels_vec):
        self.input_ids = [chunk.long() for chunk in input_ids]
        # print("length in put ids:", len(self.input_ids))
        self.attention_mask = attention_mask
        self.labels_vec = labels_vec

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        # print(self.labels_vec[index])
        return {
            'input_ids': self.input_ids[index].squeeze(),
            'attention_mask': self.attention_mask[index].squeeze(),
            'label': self.labels_vec[index]
        }

In [ ]:
print(len(total_chunks))

In [ ]:
dataset =  CustomDataset(total_chunks,total_att_mask,total_labels)

In [ ]:
len(dataset[0])

In [ ]:
# len(dataset[0]['input_ids'])
for i in dataset:
    if len(i['input_ids']) != 512: 
        print(len(i['input_ids']))

In [ ]:
len(total_labels)

In [ ]:
print(total_labels.count(0))
print(total_labels.count(1))
print(total_labels.count(2))


In [ ]:
from torch.utils.data import random_split
train_ratio = 0.8  # 80% for training
test_ratio = 0.2  # 20% for testing

num_samples = len(dataset)
train_size = int(train_ratio * num_samples)
test_size = num_samples - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


In [ ]:
len(train_dataset)

In [ ]:
len(train_dataset[3]['input_ids'])

In [ ]:
# train_data = pd.read_parquet('/home/ubuntu/working_directory/Bert_experimentation/80_train.parquet')
# val_df = pd.read_parquet('/home/ubuntu/working_directory/Bert_experimentation/80_val.parquet')
# class CustomDataset(Dataset):
#     def __init__(self, data, label2id, tokenizer):
#         self.data = data.sample(4000)
#         self.label2id = label2id
#         self.tokenizer= tokenizer
#         self.labels = self.data["labels"].apply(lambda x:self.label2id[x]).to_list()
#         self.one_hot_labels = pd.get_dummies(self.labels)
#         self.one_hot_labels = torch.tensor(np.array(self.one_hot_labels), dtype=torch.float)
#         print("Data:",self.data.labels.value_counts())
#         print("counter:",Counter(self.labels))
#         # print("@@@@",len(self.data))

#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index):
#         text = self.data.iloc[index]['texts']

#         # print("text::",index,text)
#         # print('\n')
#         label = self.one_hot_labels[index]
#         # print("original length:", len(text.split(" ")), len(text))
#         encoded_text = self.tokenizer.encode_plus(
#             str(text),
#             max_length=512,
#             add_special_tokens=True,
#             truncation=True,
#             padding='max_length',
#             return_attention_mask=True,
#             return_tensors='pt'
#         )

#         return {'input_ids': encoded_text['input_ids'].squeeze(),
#                  'attention_mask': encoded_text['attention_mask'].squeeze(),
#                  'label':label}   
                 
                 
# label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
# old_train_dataset = CustomDataset(train_data,label2id,tokenizer)
# old_test_dataset = CustomDataset(val_df,label2id,tokenizer)


In [ ]:
# len(old_train_dataset)
# print(old_test_dataset[0])

In [ ]:
len(test_dataset)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=5,
                                           pin_memory=True,
                                           shuffle=True, 
                                           )
test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=5,
                                           pin_memory=True,
                                           )

In [ ]:
train_loader

In [ ]:
# for i in train_loader:
#     print(i)

In [ ]:

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

EPOCHS = 5
LEARNING_RATE = 0.0000025 ######
BATCH_SIZE = 5
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.04) #####
# optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, weight_decay=0.04)

scheduler = get_linear_schedule_with_warmup(optimizer, 
             num_warmup_steps=50, ########
            num_training_steps=len(train_loader)*EPOCHS )   

In [ ]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")


In [ ]:
from torch.nn.utils import clip_grad_norm_
# from tqdm import tqdm
from tqdm.notebook import tqdm

import numpy as np
import math

train_loss_per_epoch = []
val_loss_per_epoch = []

model = model.to(device)
for epoch_num in range(EPOCHS):
    print('Epoch: ', epoch_num + 1)
    '''
    Training
    '''
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_loader,desc='Training')):
        # tqdm_desc = f'Training ({step_num+1}/{train_loader_length})'
        # print("batch>>>",batch_data)
        input_ids, att_mask, labels = batch_data["input_ids"].to(device),batch_data["attention_mask"].to(device),batch_data["label"].to(device)
        input_ids = input_ids.to(torch.long).to(device)
        att_mask = att_mask.to(torch.long).to(device)
        labels = labels.to(torch.long).to(device)
        # print(input_ids.shape,labels.shape)
        # input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)
        # print("logits***:", output["logits"])
        
        loss = output.loss
        train_loss += loss.item()

        model.zero_grad()
        loss.backward()
        del loss

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    train_loss_per_epoch.append(train_loss / (step_num + 1))    


    '''
    Validation
    '''
    model.eval()
    valid_loss = 0
    valid_pred = []
    with torch.no_grad():
        for step_num_e, batch_data in enumerate(tqdm(test_loader,desc='Validation')):
            
            input_ids, att_mask, labels = batch_data["input_ids"].to(device),batch_data["attention_mask"].to(device),batch_data["label"].to(device)
            # input_ids, att_mask, labels = [data.to(device) for data in batch_data]
            output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)
            # print("logits***:", output["logits"])

            loss = output.loss
            valid_loss += loss.item()

            valid_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
        
    val_loss_per_epoch.append(valid_loss / (step_num_e + 1))
    valid_pred = np.concatenate(valid_pred)
    print("{0}/{1} train loss: {2} ".format(step_num+1, math.ceil(len(train_dataset) / BATCH_SIZE), train_loss / (step_num + 1)))
    print("{0}/{1} val loss: {2} ".format(step_num_e+1, math.ceil(len(test_dataset) / BATCH_SIZE), valid_loss / (step_num_e + 1)))          


In [ ]:
import matplotlib.pyplot as plt

# Rest of your code...

epochs = range(1, EPOCHS + 1)

plt.figure(figsize=(8, 8))
plt.plot(epochs, train_loss_per_epoch, label='training loss')
plt.plot(epochs, val_loss_per_epoch, label='validation loss')
plt.title("Training and Validation Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Save the plot as a PNG file
plt.savefig('./loss_plotBig_bird_new_data.png')


In [ ]:
valid_true = [batch["label"].detach().cpu().numpy() for batch in test_loader]
valid_true = np.concatenate(valid_true)
# valid_true = np.argmax(valid_true)

In [ ]:
valid_true

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(valid_pred, valid_true,labels=[0,1,2], target_names= ["level 1","level 2","level 3"]))

In [ ]:
model.save_pretrained("./savedmodel_2")
tokenizer.save_pretrained("./savedmodel_2/")

In [ ]:
from transformers import BigBirdForSequenceClassification, BigBirdTokenizer
# Load the saved model and tokenizer
model = BigBirdForSequenceClassification.from_pretrained("./savedmodel_2/")
tokenizer = BigBirdTokenizer.from_pretrained("./savedmodel_2/")


In [ ]:
import PyPDF2
from PyPDF2 import PdfReader
def extract_text_from_pdf(file_path):
    text = ''
    with open(file_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text



In [ ]:


level_3_rows = sample_df.loc[sample_df['Level'] == 'Level 1']
print(len(level_3_rows))

In [ ]:
Data_csv


In [ ]:
sample_df

In [ ]:
new_df

In [ ]:
level_3_rows = Data_csv.loc[Data_csv['Level'] == 'Level 3']
print(len(level_3_rows['FileName']))

In [ ]:
original_data= pd.read_csv("/home/ubuntu/cat_poc/llms/data.csv")
original_data.head

In [ ]:
original_data['CE_Level'].value_counts()

In [ ]:
import pandas as pd

# Create empty lists to store the predicted labels for each row
predicted_labels = []

# Iterate over each row in the dataframe
for index, row in Data_csv.iterrows():
    print(index)
    document = row['text']
    total_words = len(document)
    chunk_size = 512
    overlap = 256
    total_chunks = (total_words - chunk_size) // (chunk_size - overlap) + 1

    chunks = []
    for i in range(total_chunks):
        start_index = i * (chunk_size - overlap)
        end_index = start_index + chunk_size
        chunk = document[start_index:end_index]
        chunks.append(chunk)

    tokenized_chunks = []
    for chunk in chunks:
        tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
        tokenized_chunks.append(tokens)

    chunk_predicted_labels = []
    for tokens in tokenized_chunks:
        outputs = model(**tokens)
        # Get the predicted class index
        predicted_class_index = outputs.logits.argmax().item()
        # Get the predicted class label
        label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
        id_to_label = {v: k for k, v in label2id.items()}
        predicted_class_label = id_to_label[predicted_class_index]
        
        chunk_predicted_labels.append(predicted_class_label)

    # Find the majority predicted label for all the chunks
    majority_label = max(set(chunk_predicted_labels), key=chunk_predicted_labels.count)
    predicted_labels.append(majority_label)
    print("True label vs predicted label",row['level'],predicted_class_label)

# Count the occurrence of each predicted label
label_counts = pd.Series(predicted_labels).value_counts()

print(label_counts)


In [ ]:

# level_3_rows = Data_csv.loc[Data_csv['Level'] == 'Level 2']
# row = level_3_rows.iloc[10]
# Random document
# document = extract_text_from_pdf("./15031-4983-FullBook.pdf")
print(row)
document = row['text']
print(len(document.split()))
# Tokenize the document
tokens = tokenizer(document, truncation=True, padding=True, return_tensors="pt")
print(tokens['input_ids'].shape)

# input_id_chunks, mask_chunks = split_tokens_into_smaller_chunks(tokens["input_ids"][0],tokens["attention_mask"][0], chunk_size, stride, minimal_chunk_length)
# add_special_tokens_at_beginning_and_end(input_id_chunks, mask_chunks)
# add_padding_tokens(input_id_chunks, mask_chunks)

# # Forward pass through the model
# outputs = model(**tokens)

# # Get the predicted class index
# predicted_class_index = outputs.logits.argmax().item()

# # Get the predicted class label
# # label2id = model.config.id2label
# label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
# id_to_label = {v: k for k, v in label2id.items()}
# predicted_class_label = id_to_label[predicted_class_index]

# print("Predicted Class:", predicted_class_label)

total_words = len(document)
chunk_size = 512
overlap = 256
total_chunks = (total_words - chunk_size) // (chunk_size - overlap) + 1

chunks = []
for i in range(total_chunks):
    start_index = i * (chunk_size - overlap)
    end_index = start_index + chunk_size
    chunk = document[start_index:end_index]
    chunks.append(chunk)
tokenized_chunks = []
for chunk in chunks:
    tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
    tokenized_chunks.append(tokens)

true_labels = []
predicted_labels = []
for i in tokenized_chunks:
    outputs = model(**tokens)
    # Get the predicted class index
    predicted_class_index = outputs.logits.argmax().item()
    # Get the predicted class label
    predicted_class_label = id_to_label[predicted_class_index]
    predicted_labels.append(predicted_class_label) 

print(len(predicted_labels))
print(predicted_labels.count('Level 1'))
print(predicted_labels.count('Level 2'))
print(predicted_labels.count('Level 3'))

In [ ]:
from sklearn.metrics import accuracy_score

# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate over the rows in the new DataFrame
for index, row in new_df.iterrows():
    document = row['text']
    print(len(document))
    true_label = row['Level']
    true_labels.append(true_label)
    # Tokenize the document
    tokens = tokenizer(document, truncation=True, padding=True, return_tensors="pt")
    print(len(tokens))
    # Forward pass through the model
    outputs = model(**tokens)
    # Get the predicted class index
    predicted_class_index = outputs.logits.argmax().item()
    # Get the predicted class label
    predicted_class_label = id_to_label[predicted_class_index]
    predicted_labels.append(predicted_class_label)

document_accuracy = accuracy_score(true_labels, predicted_labels)
print("Overall Accuracy:", document_accuracy)


In [ ]:
print(true_labels)
print(predicted_labels)

In [ ]:
lv_1_true_label = true_label[:5]
lv_2_true_label = true_label[5:10]
lv_3_true_label = true_label[10:]


lv_1_pred_label = predicted_labels[:5]
lv_2_pred_label = predicted_labels[5:10]
lv_3_pred_label = predicted_labels[10:]
print("level - 1 accuracy",accuracy_score(lv_1_true_label,lv_1_pred_label))